In [ ]:
import glob
import json
import os
import pathlib
import shutil
import zipfile

# First of all you must obtain an access token by exporting some data from Slack
INPUT_PATH = r'C:\Users\SimonTucker\BackuperyForSlack_Storage\2023_January_08-15_41_57\ReadyToImport\ONI_2023_January_08-15_44_33.zip'
OUTPUT_PATH = r'C:\Users\SimonTucker\Desktop\OUTPUT'
# INPUT_PATH = r'YOUR-SLACK-EXPORT-HERE.zip'
# OUTPUT_PATH = r'YOUR\OUTPUT\DIR\HERE'
TOKEN = os.environ.get('TOKEN')

In [ ]:
tmp_dir = os.path.join(pathlib.Path(INPUT_PATH).parent.absolute(), 'tmp')
with zipfile.ZipFile(INPUT_PATH, 'r') as zip_ref:
    zip_ref.extractall(tmp_dir)

In [ ]:
os.chdir(tmp_dir)
channel_dirs = next(os.walk('.'))[1]

In [ ]:
# Currently we only consider message type json objects
# so we perform a quick test to see if there are any other types.
# If there are other types than 'message' an investigation 
# should be performed to see what the implications of this
# might be on the operation of the code.
for channel_dir in channel_dirs:
    json_files = glob.glob(os.path.join(channel_dir, '*.json'))
    for json_file in json_files:
        with open(json_file, 'rb') as f:
            data = json.load(f)
            for obj in data:
                if obj['type'] != 'message':
                    raise Exception(f"WARNING: object of type '{obj['type']}' detected!")

In [ ]:
for channel_dir in channel_dirs:
    output_dir = os.path.join(OUTPUT_PATH, channel_dir)
    if os.path.exists(output_dir) and os.path.isdir(output_dir):
        shutil.rmtree(output_dir)
    os.mkdir(output_dir)
    json_files = glob.glob(os.path.join(channel_dir, '*.json'))
    for json_file in json_files:
        with open(json_file, 'rb') as open_json_file:
            data = json.load(open_json_file)
            for json_obj in data:
                if 'files' in json_obj:
                    for file in json_obj['files']:
                        if "url_private_download" in file:
                            link = f'{file["url_private_download"]}?t={TOKEN}'
                            file["url_private_download"] = link
        output_file = os.path.join(OUTPUT_PATH, json_file)
        with open(output_file, 'w') as open_json_file:
            json.dump(data, open_json_file, indent=4)
        

In [ ]:
# print(os.getcwd())

with open('groups.json', 'r') as f:
    string = f.read()
string = string.replace('"is_group": true', '"is_group": false')
string = string.replace('"is_private": true', '"is_private": false')

with open('groups.json', 'w') as f:
    f.write(string)

json_files = glob.glob('*.json')

for json_file in json_files:
    fname = os.path.join(tmp_dir, json_file)
    shutil.move(os.path.join(fname), OUTPUT_PATH)

os.chdir(OUTPUT_PATH)
shutil.rmtree(tmp_dir)
